In [ ]:
# HIDDEN
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 7)
pd.set_option('display.max_columns', 8)
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# HIDDEN
def df_interact(df, nrows=7, ncols=7):
    '''
    Outputs sliders that show rows and columns of df
    '''
    def peek(row=0, col=0):
        return df.iloc[row:row + nrows, col:col + ncols]

    row_arg = (0, len(df), nrows) if len(df) > nrows else fixed(0)
    col_arg = ((0, len(df.columns), ncols)
               if len(df.columns) > ncols else fixed(0))
    
    interact(peek, row=row_arg, col=col_arg)
    print('({} rows, {} columns) total'.format(df.shape[0], df.shape[1]))

def display_df(df, rows=pd.options.display.max_rows,
               cols=pd.options.display.max_columns):
    with pd.option_context('display.max_rows', rows,
                           'display.max_columns', cols):
        display(df)

In [ ]:
at = (pd.read_csv('attendance_private.csv',
                  names=['time', 'email', 'name', 'pid', 'sec', 'num'],
                  skiprows=1)
      .assign(pid=lambda at: at['pid'].str.upper())
      .assign(time=lambda at: pd.to_datetime(at['time']))
      .assign(week=lambda at: at['time'].dt.week - 2)
     )
at

In [ ]:
df_interact(at[
    (at['email'].str.contains('007'))
    & (at['sec'].isin(['Wed 12pm', 'Wed 1pm']))
])

In [ ]:
students = (pd.pivot_table(at.assign(at=1), values='at', index='pid', columns='week')
 .fillna(0)
 .astype(int)
)
students

In [ ]:
(at.groupby('week')
 .size()
 .rename('checkins')
 .to_frame()
)

In [ ]:
plt.figure(figsize=(10, 10))
(at.set_index('time')
 .resample('1D')
 .size()
 .rename('checkins')
 .to_frame()
 .query('checkins > 0')
 .plot.line()
)

In [ ]:
plt.figure(figsize=(10, 10))
(at.groupby(['week', 'sec'])
 .size()
 .plot.barh()
)